In [0]:
%sql 
select *from pyspark_cata.source.products 

id,name,price,catagory,updatedDate
1,iPhone,1000,electronics,2025-11-23T00:30:11.517Z
2,Macbook,2000,electronics,2025-11-23T00:30:11.517Z
3,Airpods,500,electronics,2025-11-23T00:30:11.517Z
4,Shirt,100,clothing,2025-11-23T00:30:11.517Z
5,Pants,150,clothing,2025-11-23T00:30:11.517Z
5,Trousers,150,clothing,2025-11-24T21:09:42.783Z


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [0]:
class upsertwithdedup:
    def __init__(self, df):
        self.df = df 
    

    def dedup(self, keyCol, cdcCol):
        df = self.df.withColumn("dedup", row_number().over(window.partitionBy(keyCol).orderBy(desc(cdcCol))))
        df = df.filter(col("dedup"==1).drop("dedup"))
        #replace stored df
        self.df = df
        return df
    
    def table_exists(self, spark, targetPath):
        try:
            DeltaTable.forPath(spark, targetPath)
            return True
        except Exception as e:
            return False
    def created_delta_table(self, spark, tartgetPath):
        (
            self.df.write
            .format("delta")
            .mode("overwrite")
            .option("mergeSchema", "true")
            .save(targetPath)

        )
    print("New Delta Table Created with schema")

    
    def upsert(self, spark, targetPath, keyCol, cdcCol): 
        if not self.table_exists(spark, targetPath):
            print("Table does not exist, creating new table")
            self.created_delta_table (spark, targetPath)
            return
        print("Valid Delta table found -> permorning MERGE...")

        dlt_obj = DeltaTable.forPath(spark, targetPath)
        (
            dlt_obj.alias("trg")
            .merge(self.df.alias("src")),
            f"src.{keyCol} = trg.{keyCol}"
        )\
        .whenMatchedUpdateAll(condition= f"src.{cdcCol} >=trg.{cdcCol}")\
        .whenNotMatchedInsertAll()\
        .execute()
        print("Merge completed")


 

New Delta Table Created with schema


In [0]:
#loading data frame
df = spark.table("pyspark_cata.source.products")


In [0]:
#creating class oject
obj = upsertwithdedup(df)


In [0]:
df_dedup = obj.dedup("id", "updatedDate")


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-8280771191677814>, line 1
----> 1 df_dedup = obj.dedup("id", "updatedDate")

File <command-8280771191677809>, line 7, in upsertwithdedup.dedup(self, keyCol, cdcCol)
      6 def dedup(self, keyCol, cdcCol):
----> 7     df = self.df.withColumn("dedup", row_number().over(window.partitionBY(keyCol).orderBy(desc(cdcCol))))
      8     df = df.filter(col("dedup"==1).drop("dedup"))
      9     #replace stored df

AttributeError: 'function' object has no attribute 'partitionBY'